In [1]:
import networkx as nx
import pandas as pd
import numpy as np
from sklearn.feature_selection import mutual_info_classif
from pathlib import Path
from frlearn.classifiers import FRNN
from sklearn.model_selection import train_test_split
from frlearn.base import probabilities_from_scores
from sklearn.metrics import roc_auc_score
from frlearn.neighbours.instance_preprocessors import FRPS
import time
from frlearn.neighbours.data_descriptors import NND

from frlearn.base import DataDescriptor, MultiClassClassifier, MultiLabelClassifier
from frlearn.data_descriptors import NND
from frlearn.neighbour_search_methods import NeighbourSearchMethod, KDTree
from frlearn.neighbours.utilities import resolve_k
from frlearn.statistics.feature_preprocessors import RangeNormaliser
from frlearn.array_functions import div_or, soft_max, soft_min
from frlearn.parametrisations import at_most, multiple
from frlearn.transformations import truncated_complement
from frlearn.uncategorised.utilities import resolve_dissimilarity
from frlearn.weights import ExponentialWeights, LinearWeights



from Ant import Colony as Colony1
from Ant import Colonies
from Main import Colony as Colony


In [2]:
index = ["bkblk","bknwy","bkon8","bkona","bkspr","bkxbq","bkxcr","bkxwp","blxwp","bxqsq","cntxt","dsopp","dwipd",
         "hdchk","katri","mulch","qxmsq","r2ar8","reskd","reskr","rimmx","rkxwp","rxmsq","simpl","skach","skewr",
         "skrxp","spcop","stlmt","thrsk","wkcti","wkna8","wknck","wkovl","wkpos","wtoeg","win"]


data_path = Path.home().joinpath('projects/rough_set/kr-vs-kp_csv.csv').as_posix()
data = pd.read_csv(data_path)
data.rename(columns = {'class':"win"}, inplace=True)


ns = pd.get_dummies(data[data.columns[:-1]], prefix=data.columns[:-1],drop_first=True, dtype=int)

ns_y = data['win'].copy()
Y = pd.get_dummies(ns_y,prefix='win',drop_first=True, dtype=int)


In [3]:
data = ns.iloc[:, [9,21,33]]
X_train, X_test, y_train, y_test = train_test_split(data.values, Y.values.squeeze(), test_size=0.33, random_state=42)
clf = FRNN()
X = ns.iloc[:, [9,21,33]].values
y = Y.values.squeeze()
model = clf(X_train, y_train)
pred = model(X_test)
    

In [5]:
pred.shape

(1055, 2)

In [4]:
l0 = model.lower_approximations[0]
l1 = model.lower_approximations[1]
# l.query

In [8]:
model.lower_approximations[0]._query

<bound method NND.Model._query of <frlearn.neighbours.data_descriptors.NND.Model object at 0x7f7e99762500>>

In [9]:
l0._query

<bound method NND.Model._query of <frlearn.neighbours.data_descriptors.NND.Model object at 0x7f7e99762500>>


In [23]:
features = [
    [5,25,27,1],
    [9,21,33],
    [5,0,3,7,14],
    [4,5,21,31]
]


for feature in features:
    data = ns.iloc[:, feature]
    
    X_train, X_test, y_train, y_test = train_test_split(data.values, Y.values.squeeze(), test_size=0.33, random_state=42)
    clf = FRNN()
    X = ns.iloc[:, feature].values
    y = Y.values.squeeze()
    model = clf(X_train, y_train)
    co_Cs = [X[np.where(y != c)] for c in [0,1]]
    # model_query = [model._query(co_C) for co_C in co_Cs]
    
    print(f'preprocessing model : {model_query}')
   
    # model_nnd = nnd(X)
    lower_approximator = NND()
    
    lower_approximations = [lower_approximator(co_C) for co_C in co_Cs]
   
    scores = model(X_test)
    probabilities = probabilities_from_scores(scores)
    auroc = roc_auc_score(y_test, probabilities[:,-1])
    print(f'features --> {feature}---auroc --> {auroc}\n')
    print(f'lower-approximation --> {lower_approximations}')

            
# co_Cs = [ns[np.where(Y != c)] for c in [0,1]]
# model.lower_approximations = self.lower_approximator and [self.lower_approximator(co_C) for co_C in co_Cs]
        

preprocessing model : [array([[0.5  , 0.5  ],
       [0.5  , 0.5  ],
       [0.5  , 0.5  ],
       ...,
       [0.375, 0.625],
       [0.375, 0.625],
       [0.375, 0.625]]), array([[0.5, 0.5],
       [0.5, 0.5],
       [0.5, 0.5],
       ...,
       [0.5, 0.5],
       [0.5, 0.5],
       [0.5, 0.5]])]
features --> [5, 25, 27, 1]---auroc --> 0.514020174053485

lower-approximation --> [<frlearn.neighbours.data_descriptors.NND.Model object at 0x7f94523be290>, <frlearn.neighbours.data_descriptors.NND.Model object at 0x7f94fbfa6fb0>]
preprocessing model : [array([[0.5  , 0.5  ],
       [0.5  , 0.5  ],
       [0.5  , 0.5  ],
       ...,
       [0.375, 0.625],
       [0.375, 0.625],
       [0.375, 0.625]]), array([[0.5, 0.5],
       [0.5, 0.5],
       [0.5, 0.5],
       ...,
       [0.5, 0.5],
       [0.5, 0.5],
       [0.5, 0.5]])]
features --> [9, 21, 33]---auroc --> 0.8202937422941317

lower-approximation --> [<frlearn.neighbours.data_descriptors.NND.Model object at 0x7f945237ceb0>, <frlea

In [49]:
X = ns.iloc[:, [9,21,33]].values
y = Y.values.squeeze()
upper_weights = LinearWeights()
upper_k = at_most(20)
lower_weights = LinearWeights()
lower_k = at_most(20)
dissimilarity = 'boscovich'
nn_search = KDTree()
preprocessors = RangeNormaliser()
lower_approximator = NND(weights=lower_weights, k=lower_k,
                         proximity=truncated_complement,
                         dissimilarity=dissimilarity,
                         nn_search=nn_search, preprocessors=())
# lower_approximator = NND()
co_Cs = [X[np.where(y != c)] for c in [0,1]]
lower_approximations = [lower_approximator(co_C) for co_C in co_Cs]  
lower_approximations[1]._query(X)

TypeError: NND.Model._query() missing 1 required positional argument: 'q_distances'

[array([[0, 0, 0, 0],
        [1, 0, 0, 0],
        [1, 0, 0, 0],
        ...,
        [0, 0, 1, 0],
        [0, 1, 1, 0],
        [0, 1, 1, 0]]),
 array([[0, 0, 0, 0],
        [1, 0, 0, 0],
        [0, 0, 0, 0],
        ...,
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0]])]

In [ ]:
m.lower_approximator()

In [2]:
main_colony = Colony()
main_colony2 = Colony()

In [3]:
main_colony2.pheromone_traversed()

for insure pheromone value before changing: [[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1.

In [8]:
# colonies3 = Colonies()
# main_colony2.initialize_colony(colonies=colonies3,
#                                number_of_ants_first_generation=5,
#                                init_criteria=10,
#                                q=0.8,
#                                phero_rate_decay=0.5,
#                                change_pheromone=False,
#                                with_core=True,
#                                core_accuracy=0.75) 

# main_colony2.generate_next_ants(colonies=colonies3,
#                                 number_of_ants_next_generation=4,
#                                 q=0.8,
#                                 phero_rate_decay=0.5,
#                                 THRESHOLD=[0.98, 'accuracy', 0.80],
#                                 alpha=0.2,
#                                 beta=0.001,
#                                 with_core=False,
#                                 change_pheromone=True)
## with_core --> at least 1 and max 4
## THRESHOLD --> at least 1 and max 4
## remaining variables --> at least 2 max 5
main_colony2.generation(number_of_ants=[3,1],
                        init_criteria=5,
                        number_of_generations=4,
                        q=[0.8,0.9],
                        phero_rate_decay=[0.5,0.5],
                        THRESHOLD=[[0.97, 'accuracy', 0.80],
                                   [0.975, 'accuracy', 0.80],
                                   [0.98, 'accuracy', 0.80],
                                   [0.995, 'accuracy', 0.80]],
                        alpha=0.2,
                        beta=0.001,
                        with_core=[True,False,True],
                        change_pheromone=[True,True,True,False],
                        core_accuracy=0.75)

Error: Enter a Value between 2 and less
than or equall 5
for variable q


In [6]:
main_colony2.route_data

{'initialization': {'ant-0': [6, 10, 3, 24, 21],
  'ant-1': [13, 1, 35, 18, 9],
  'ant-2': [31, 3, 1, 30, 19]},
 'generation': {'generation-1': {'ant-0': [9,
    32,
    7,
    15,
    36,
    29,
    31,
    16,
    18,
    6,
    23,
    14,
    35,
    1,
    27,
    22,
    5,
    0,
    2,
    3,
    4,
    8,
    10,
    11,
    12,
    13,
    17,
    19,
    20,
    24,
    25,
    26,
    28,
    30,
    34]},
  'generation-2': {'ant-0': [25,
    21,
    9,
    32,
    7,
    15,
    36,
    29,
    31,
    16,
    18,
    6,
    23,
    14,
    35,
    1,
    27,
    22,
    5,
    33]},
  'generation-3': {'ant-0': [9,
    32,
    7,
    15,
    36,
    29,
    31,
    16,
    18,
    6,
    23,
    14,
    35,
    1,
    27,
    22,
    5,
    0,
    2,
    3,
    4,
    8,
    10,
    11,
    12,
    13,
    17,
    19,
    20,
    24,
    25,
    26,
    28,
    30,
    34]},
  'generation-4': {'ant-0': [21,
    32,
    7,
    15,
    36,
    29,
    31,
    16,
    18,
  

In [8]:
# Colonies.pheromone
main_colony3 = Colony()
# main_colony2.pheromone_traversed()
for row in main_colony2.route_data['generation'].values():
    print(row)
# main_colony2.core_feature
# main_colony.traversed_nodes


{'ant-0': [33, 32, 7, 15, 36, 29, 31, 16, 18, 6, 23, 14, 35, 1, 27, 22, 5, 0, 2, 3, 4, 8, 10, 11, 12, 13, 17, 19, 20, 24, 25, 26, 28, 30, 34]}


In [9]:
main_colony2.generate_next_ants(number_of_ants_next_generation=2,
                          q=0.8,
                          phero_rate_decay=0.5,
                          THRESHOLD=[0.75, 'accuracy', 0.80],
                          alpha=0.2,
                          beta=0.001,
                          with_core=True,
                          change_pheromone=True)

for sure!! must be empty: []
self.fg_core in ant class: [0, 1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 34, 35, 36]
self.fg_core in ant class: [0, 1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 34, 35, 36]
self.fg_core in ant class: [0, 1, 2, 3, 4, 5, 6, 8, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 34, 35, 36]
self.fg_core in ant class: [0, 1, 2, 3, 4, 5, 6, 8, 10, 11, 12, 13, 14, 16, 17, 18, 19, 20, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 34, 35, 36]
self.fg_core in ant class: [0, 1, 2, 3, 4, 5, 6, 8, 10, 11, 12, 13, 14, 16, 17, 18, 19, 20, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 34, 35]
self.fg_core in ant class: [0, 1, 2, 3, 4, 5, 6, 8, 10, 11, 12, 13, 14, 16, 17, 18, 19, 20, 22, 23, 24, 25, 26, 27, 28, 30, 31, 34, 35]
self.fg_core in ant class: [0, 1, 2, 3, 4, 5, 6, 8, 10, 11, 12, 13, 14, 16, 17, 18, 19, 20, 22, 2

In [1]:
import pmlb

In [10]:
main_colony.pheromone_traversed()

for insure pheromone value before changing: [[0.25       0.25       0.25       0.25       0.25       0.25
  0.25       0.25       0.25       0.25       0.25       0.25
  0.25       0.25       0.25       0.25       0.25       0.25
  0.25       0.25       0.25       0.25       0.25       0.25
  0.25       0.25       0.25       0.25       0.25       0.25
  0.25       0.25       0.25       0.25       0.25       0.25
  0.25      ]
 [0.25       0.25       0.25       0.25       0.25       0.25
  0.25       0.25       0.25       0.25       0.25       0.25
  0.25       0.25       0.25       0.25       0.25       0.25
  0.25       0.25       0.25       0.25       0.25       0.25
  0.25       0.25       0.25       0.33820513 0.25       0.25
  0.25       0.25       0.25       0.25       0.25       0.25
  0.25      ]
 [0.25       0.25       0.25       0.25       0.25       0.25
  0.25       0.25       0.25       0.25       0.25       0.25
  0.25       0.25       0.25       0.25       0.25       0.2

In [15]:
print(colonies.traversed_nodes)

[False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False]


In [4]:
main_colony.initialize_colony(5, 3, with_core=True, core_accuracy=0.75) 

main_colony.generate_next_ants(number_of_ants_next_generation=2,
                          q=0.8,
                          phero_rate_decay=0.5,
                          THRESHOLD=[0.95, 'accuracy', 0.80],
                          alpha=0.8,
                          beta=0.9,
                          with_core=True,
                          change_pheromone=True)

9
21
33
[9, 21, 33]
ant_route: [22, 6, 21]
ant_instance.fg: [0, 1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36]
ant_route: [20, 17, 10]
ant_instance.fg: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 12, 13, 14, 15, 16, 18, 19, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36]
ant_route: [21, 26, 23]
ant_instance.fg: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 22, 24, 25, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36]
ant_route: [18, 14, 33]
ant_instance.fg: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15, 16, 17, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 34, 35, 36]
ant_route: [12, 13, 7]
ant_instance.fg: [0, 1, 2, 3, 4, 5, 6, 8, 9, 10, 11, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36]
initiall final: [0, 1, 2, 3, 4, 5, 6, 8, 9, 10, 11, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 

In [ ]:
print(f'initial : {Colonies.overall_ant_route_init}')
print(f'next GEN : {Colonies.overall_ant_route}')

In [4]:
ant_colony1.initialize_colony(number_of_ants_first_generation=5,
                              init_criteria=3,q=0.8,
                              phero_rate_decay=0.5,
                              make_change_pheromone=False,
                              with_core=True,
                              core_accuracy=0.75)
ant_colony1.generate_next_ants(number_of_ants_next_generation=5,
                          q=0.8,
                          phero_rate_decay=0.5,
                          THRESHOLD=[0.97, 'accuracy', 0.80],
                          alpha=0.8,
                          beta=0.4,
                          with_core=True,
                          change_pheromone=True)

9
21
33
[9, 21, 33]
ant_route: [26, 0, 4]
ant_instance.fg: [1, 2, 3, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36]
ant_route: [11, 33, 7]
ant_instance.fg: [0, 1, 2, 3, 4, 5, 6, 8, 9, 10, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 34, 35, 36]
ant_route: [28, 1, 18]
ant_instance.fg: [0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 19, 20, 21, 22, 23, 24, 25, 26, 27, 29, 30, 31, 32, 33, 34, 35, 36]
ant_route: [36, 19, 2]
ant_instance.fg: [0, 1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35]
ant_route: [3, 31, 7]
ant_instance.fg: [0, 1, 2, 4, 5, 6, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 32, 33, 34, 35, 36]
initiall final: [0, 1, 2, 4, 5, 6, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 32, 33, 34, 35,

In [ ]:
print(f'initial : {Colonies.overall_ant_route_init}')
print(f'next GEN : {Colonies.overall_ant_route}')

In [ ]:
np.set_printoptions(threshold=np.inf)
print(f'pheromone : {Colonies.pheromone}') 

# transfer to GAN

In [ ]:
D = Lf*(1 + landa)
Lf = max(ant_route accuracy)
G = Li
if Li < D:
    change_phromone()

In [ ]:
len(ns)

In [ ]:

X = ns.iloc[:, [36, 1, 2]].values
y = Y.values.squeeze()
clf = FRPS()
## FRPS() choose instances that are not ROUGH!
## instances with quality measure more than maximum tau
selected_dataset = clf(X, y)
print(len(selected_dataset[0])/len(ns))
# len(selected_dataset[1])

In [ ]:
len(selected_dataset[0])

In [ ]:
X = ns.iloc[:, [32]].values
y = Y.values.squeeze()
clf = FRPS()
## FRPS() choose instances that are not ROUGH!
## instances with quality measure more than maximum tau
selected_dataset = clf(X, y)
len(selected_dataset)/len(X)
len(selected_dataset[0])
# round(625 / len(X),3)

In [ ]:
selected_dataset[]
len(selected_dataset[1])

In [ ]:
Colony.log